In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from keras.layers import Activation
from tensorflow.keras.initializers import RandomNormal
from sklearn.model_selection import train_test_split
import xgboost as xgb
from scipy import integrate
from scipy import signal
from tensorflow import keras
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.layers import Input, LSTM, Dense, Dropout, Reshape, Flatten, concatenate, Dot, Concatenate, Lambda
import keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping
import seaborn as sns
sns.set_style('whitegrid')
sns.set_context("notebook")
sns.set_palette("husl")
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.layers import Input, LSTM, Flatten, Dense, Conv1D, Dropout, MaxPooling1D
from deap import base, creator, tools, algorithms
from scipy.stats import bernoulli
from bitstring import BitArray
from keras.models import Model
from sklearn.metrics import mean_squared_error
from keras.layers import Activation

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)

print(tf.__version__)

print(tf.config.list_physical_devices('GPU'))

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
def select_df(df):
    mag=df['Magnitude']
    selected_features=["RSSC Vel",
                   "CAV","TP",
                   "Tau c","Max Acc Index UD","IV_2",
                           "Cheby Tau c","Arias intensity UD",
                   "Cheby IV_2","CAD","Cheby RSSC Vel"]
    df=df[selected_features]
    return df,mag
def create_image_dataset():
    df=pd.read_csv("E:/ICRAGEE 2024/Result India/Test_tab_MAG_data_3C.csv",sep=',')
    tab_test,y_test=select_df(df)
    df=pd.read_csv("E:/ICRAGEE 2024/Result India/Val_tab_MAG_data_3C.csv",sep=',')
    tab_val,y_val=select_df(df)
    df=pd.read_csv("E:/ICRAGEE 2024/Result India/Train_tab_MAG_data_3C.csv",sep=',')
    tab_train,y_train=select_df(df)
    seismic_test = np.load('E:/ICRAGEE 2024/Result India/Test_waveform_MAG_data_3C.npy')
    seismic_train = np.load('E:/ICRAGEE 2024/Result India/Train_waveform_MAG_data_3C.npy')
    seismic_val = np.load('E:/ICRAGEE 2024/Result India/Val_waveform_MAG_data_3C.npy')
    return y_train,y_test,y_val, seismic_train, seismic_test, seismic_val,tab_train,tab_test, tab_val

# Create the image dataset
y_train,y_test,y_val, seismic_train, seismic_test, seismic_val,tab_train,tab_test, tab_val=create_image_dataset()
print("X train size:",y_train.shape)
print("X test size:", y_test.shape)
print("X validation size:",y_val.shape)
print("X waveform validation size:",seismic_val.shape)
def change_to_tensor(arr):
    x_tensor = tf.convert_to_tensor(arr.astype(np.float32))
    return x_tensor
y_train=change_to_tensor(y_train)
y_test=change_to_tensor(y_test)
y_val=change_to_tensor(y_val)
tab_test=change_to_tensor(np.array(tab_test))
tab_train=change_to_tensor(np.array(tab_train))
tab_val=change_to_tensor(np.array(tab_val))

In [ ]:
# Train model on GPU
with tf.device('/GPU:0'):
    def train_evaluate(ga_individual_solution):  
        num_units_bits_LSTM1 = BitArray(ga_individual_solution[0:6]) 
        num_units_bits_LSTM2 = BitArray(ga_individual_solution[6:]) 
        num_units_bits_LSTM1 = num_units_bits_LSTM1.uint
        num_units_bits_LSTM2 = num_units_bits_LSTM2.uint
        print('\nNum of Units LSTM 1: ', num_units_bits_LSTM1, ', Num of Units LSTM 2: ', num_units_bits_LSTM2)
        if num_units_bits_LSTM1 == 0 or num_units_bits_LSTM2 == 0:
            return 32, 
        your_target_height = 128
        your_target_width = 128
        your_sequence_length = 300 
        your_num_features = 3
        lstm_input = Input(shape=(your_sequence_length, your_num_features)) 
        lstm_layer_1 = LSTM(num_units_bits_LSTM1,return_sequences=True)(lstm_input)
        lstm_layer_2 = LSTM(num_units_bits_LSTM2)(lstm_layer_1)
        flatten_layer = Flatten()(lstm_layer_2)
        dense1 = Dense(64, activation='relu')(flatten_layer)
        predicted_values = Dense(1, activation='linear')(dense1)
        model = Model(inputs=[lstm_input], outputs=predicted_values)
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate=0.0001,
                decay_steps=10,
                decay_rate=0.96
            )
        optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)
        model.compile(optimizer=optimizer,loss='mean_squared_error')
        model.fit(x=seismic_train, y=y_train, batch_size=32, epochs=100, 
              validation_data=(seismic_val, y_val))
        y_pred = model.predict(seismic_test)   
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        print('Validation RMSE: ', rmse,'\n')
        return rmse,

In [ ]:
population_size = 4
num_generations = 4
gene_length = 10
creator.create('FitnessMax', base.Fitness, weights = (-1.0,))
creator.create('Individual', list , fitness = creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register('binary', bernoulli.rvs, 0.5)
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.binary, n = gene_length)
toolbox.register('population', tools.initRepeat, list , toolbox.individual)
toolbox.register('mate', tools.cxOrdered)
toolbox.register('mutate', tools.mutShuffleIndexes, indpb = 0.6)
toolbox.register('select', tools.selRoulette)
toolbox.register('evaluate', train_evaluate)
population = toolbox.population(n = population_size)
r = algorithms.eaSimple(population, toolbox, cxpb = 0.4, mutpb = 0.1, ngen = num_generations, verbose = False)

In [ ]:
optimal_individuals_data = tools.selBest(population,k = 1) #select top 1 solution
optimal_window_size = None
optimal_num_units = None

for bi in optimal_individuals_data:
    window_size_bits = BitArray(bi[0:6])
    num_units_bits = BitArray(bi[6:]) 
    num_units_bits_LSTM1 = window_size_bits.uint
    num_units_bits_LSTM2 = num_units_bits.uint
    print('\n Best Num of Units LSTM 1: ', num_units_bits_LSTM1, ', Best Num of Units LSTM 2: ', num_units_bits_LSTM2)

In [ ]:
with tf.device('/GPU:0'):
    def create_attention_lstm(target_height, target_width, your_sequence_length, your_num_features,num_units_bits_LSTM1,num_units_bits_LSTM2):
        lstm_input = Input(shape=(your_sequence_length, your_num_features)) 
        lstm_layer_1 = LSTM(num_units_bits_LSTM1,return_sequences=True)(lstm_input)
        lstm_layer_2 = LSTM(num_units_bits_LSTM2)(lstm_layer_1)
        flatten_layer = Flatten()(lstm_layer_2)
        dense1 = Dense(12, activation='relu')(flatten_layer)
        output_layer = Dense(1, activation='linear')(dense1)
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate=0.0001,
                decay_steps=10,
                decay_rate=0.96
            )
        optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)
        model_cnn_bilstm = Model(inputs=[lstm_input], outputs=output_layer)
        model_cnn_bilstm.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
        return model_cnn_bilstm

    your_target_height = 128
    your_target_width = 128
    your_sequence_length = 300 
    your_num_features = 3
    model = create_attention_lstm(your_target_height, your_target_width, your_sequence_length, your_num_features,num_units_bits_LSTM1,num_units_bits_LSTM2)
    checkpoint = ModelCheckpoint(filepath='Attention_weights.hdf5', save_best_only=True, monitor='val_mae', mode='min')
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(x=seismic_train, y=y_train, batch_size=32, epochs=100, 
              validation_data=(seismic_val, y_val),callbacks=[early_stopping,checkpoint])
    model.load_weights('Attention_weights.hdf5')

In [ ]:
model.save('Model_Magnitude.h5')
y_pred = model.predict([seismic_test])
y_pred_train = model.predict([seismic_train])
xgb_model = xgb.XGBRegressor(max_depth=3,learning_rate=0.1,n_estimators=230,subsample=0.7,seed=1,
                            alpha=0.1,min_child_weight=1,random_state=1,scale_pos_weight = 1,base_score=300,
                            reg_lambda=0.2 )
combined_data = np.concatenate((tab_train,y_pred_train), axis=1)
xgb_model.fit(combined_data, y_train)
combined_data = np.concatenate((tab_test, y_pred ), axis=1)
y_pred_test=xgb_model.predict(combined_data)

mae = mean_absolute_error(y_test,y_pred_test)
print("Mean Absolute Error:", mae)

plt.scatter(y_test,y_pred_test,s=10,color='black')
plt.xlim(2, 8)
plt.ylim(2, 8)    